<a href="https://colab.research.google.com/github/Sagar2402/IECSE-ML-Summer-2021/blob/AkshatBhandari-1/Titanic_Dataset_DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import sys
import functools

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

In [3]:
import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

In [14]:
titanic_file_path = tf.keras.utils.get_file("train.csv", "https://storage.googleapis.com/tf-datasets/titanic/train.csv")
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)


16384/13049 [=====================================] - 0s 0us/step


In [22]:
titanic_train_set = tf.data.experimental.make_csv_dataset(
    titanic_file_path,
    batch_size=12,
    label_name='survived',
    num_epochs=1,
    ignore_errors=True)

titanic_test_set = tf.data.experimental.make_csv_dataset(
      test_file_path,
      batch_size=12, 
      label_name='survived',
      na_value="?",
      num_epochs=1,
      ignore_errors=True)


In [7]:
for batch, label in titanic_csv_ds.take(1):
  for key, value in batch.items():
    print(f"{key:20s}: {value}")
  print()
  print(f"{'label':20s}: {label}")

sex                 : [b'male' b'male' b'female' b'male' b'male' b'female' b'male' b'male'
 b'male' b'female' b'female' b'female']
age                 : [10.  28.  44.  28.  55.5 14.  14.  39.  19.  28.  14.  28. ]
n_siblings_spouses  : [3 0 0 0 0 1 5 0 0 1 1 0]
parch               : [2 0 1 0 0 0 2 0 0 0 0 0]
fare                : [27.9    13.8625 57.9792  7.8958  8.05   11.2417 46.9    13.      8.05
 15.5    30.0708  7.2292]
class               : [b'Third' b'Second' b'First' b'Third' b'Third' b'Third' b'Third' b'Second'
 b'Third' b'Third' b'Second' b'Third']
deck                : [b'unknown' b'unknown' b'B' b'unknown' b'unknown' b'unknown' b'unknown'
 b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Cherbourg' b'Cherbourg' b'Southampton' b'Southampton'
 b'Cherbourg' b'Southampton' b'Southampton' b'Southampton' b'Queenstown'
 b'Cherbourg' b'Cherbourg']
alone               : [b'n' b'y' b'n' b'y' b'y' b'n' b'n' b'y' b'y' b'n' b'n' b'y']

lab

In [18]:
CATEGORIES = {
    'sex': ['male', 'female'],
    'class' : ['First', 'Second', 'Third'],
    'deck' : ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J'],
    'embark_town' : ['Cherbourg', 'Southhampton', 'Queenstown'],
    'alone' : ['y', 'n']
}
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [19]:
def process_continuous_data(mean, data):
  # Normalize data
  data = tf.cast(data, tf.float32) * 1/(2*mean)
  return tf.reshape(data, [-1, 1])

MEANS = {
    'age' : 29.631308,
    'n_siblings_spouses' : 0.545455,
    'parch' : 0.379585,
    'fare' : 34.385399
}

numerical_columns = []

for feature in MEANS.keys():
  num_col = tf.feature_column.numeric_column(feature, normalizer_fn=functools.partial(process_continuous_data, MEANS[feature]))
  numerical_columns.append(num_col)

preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numerical_columns)

In [30]:
model = tf.keras.Sequential([
  preprocessing_layer,
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy'])

In [31]:
train_data = titanic_train_set.shuffle(500)
test_data = titanic_test_set
model.fit(train_data, epochs=20)

Epoch 1/20
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
53/53 [==============================] - 1s 2ms/step - loss: 0.5376 - accuracy: 0.7352
Epoch 2/20
53/53 [==============================] - 0s 2ms/step - loss: 0.4436 - accuracy: 0.8054
Epoch 3/20
53/53 [==============================] - 0s 3ms/step - loss: 0.4188 - accuracy: 0.8182
Epoch 4/20
53/53 [==============================] - 0s 2ms/step - loss: 0.4032 - accuracy: 0.8262
Epoch 5/20
53/53 [==============================] - 0s 2ms/step - loss: 0.4008 - accuracy: 0.8214
Epoch 6/20
53/53 [==============================] - 0s 2ms/step - loss: 0.3880 - accuracy: 0.8150
Epoch 7/20
53/53 [==============================] - 0s 3ms/step - loss: 0.3813 - accuracy: 0.8373
Epoch 8/20
53/53 [==============================] - 0s 2ms/step - loss: 0.3745 - accuracy: 0.8437
Epoch 9/20
53/53 [==============================] - 0s 2ms/step - loss: 0.3769 - accuracy: 0.8373
Epoch 10

In [32]:
test_loss, test_accuracy = model.evaluate(test_data)


Consider rewriting this model with the Functional API.
22/22 [==============================] - 0s 2ms/step - loss: 0.4436 - accuracy: 0.8144
